# Comparison of GC of cephalopods in Ensembl or NCBI releases
# batch 3 - with spaces and "N" in genome with different color than cmap

<font color='red'>**For production**</font>

<font color='red'>**New version for Cephalopods**</font>

all done in batch - data are prepared by **batch_for_cephalophod_GC_repeats_profile_PLOS.py** in the same directory

Marta Vohnoutová, Radka Symonová

version 0.3

Changes:
    
    - dots in graphs - same density - adjasted to left
    - pick up chromosomes only - universal
    - with spaces and "N" in genome with different color than cmap

## <font color='red'>2.Import section</font>

In [23]:
#import pyensembl

from Bio import SeqIO
from Bio.SeqUtils import gc_fraction
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.ticker import FormatStrFormatter, StrMethodFormatter
from matplotlib.ticker import MaxNLocator
from matplotlib import font_manager, cm
plt.rcParams["figure.figsize"] = [20.00, 20.00]
import pandas as pd
import numpy as np
import os
import csv
from collections import OrderedDict
from PIL import Image
from glob import glob
import pickle

## <font color='red'>3.Configuration and download section</font>

In [24]:
!pwd

/mnt/data/marta/evan/PlosComp/dataOIST


# save what_to_operate for next usage

In [25]:
# read what_to_operate from pickle

file = open('/home/marta/marta/marta/evan/PlosComp/dataOIST/pickles/what_to_operate_Prochlorococcus_marinus.pkl','rb')
what_to_operate = pickle.load(file)
file.close()

In [26]:
what_to_operate

[['/home/marta/marta/marta/evan/PlosComp/dataOIST/Ansgar/Prochlorococcus_marinus/marinus_Anasgar.fasta',
  'Prochlorococcus_marinus',
  [1000],
  'rec.id.startswith("NZ")',
  'Ansgar']]

### 3.1 Variable part - replace with right values

In [27]:
where_i_am = '/mnt/data/marta/evan/PlosComp/dataOIST/' # replace with your path
column_name = 'GC DNA' # replace with your column name

#### Create subdirectory for particular species

# DNA only relese ..... - pandas table

## GC DNA release ..... as a batch in cycle

In [28]:
def how_many(my_seq,string):
    my_sum=0
    for base in string:
        my_sum += my_seq.count(base)
    return my_sum

In [29]:
# Constants for graph
column_name = 'GC DNA' # replace with your column name
# Set a special value that should be colored differently
special_value = 0.0

bottom = cm.get_cmap('RdYlGn', 255)
special_color = (0.0,0.0,0.999,0.999)
newcolors = np.vstack((special_color,bottom(np.linspace(0, 1, 255))))
new_cmap = ListedColormap(newcolors, name='new_RdYlGn')

slight_move=0.009         

In [30]:
for my_fasta_file in what_to_operate:  #highest level
    for my_window in my_fasta_file[2]:
        print(my_fasta_file[0],my_window)
        sec_values = '' # replace with your release number
        c=[]

        GC_chromosoms=OrderedDict()
        chomosome_no=0

        gc_values=tuple()

        d_sec_values = dict()

        max_len=max([len(rec.seq) for rec in SeqIO.parse(f"{my_fasta_file[0]}", "fasta")]) # max len to adjust x axis

        for rec in SeqIO.parse(f"{my_fasta_file[0]}", "fasta"): 
            GC_chromosoms_windowed={}  # per chromosome
            if eval(my_fasta_file[3]):  # chromosomes only
        
                # count GC values via sliding windows
                c.append(rec.id)
                sec_values += rec.seq
                if len(sec_values) >= 10*my_window:
                    gc_values = gc_values + tuple(gc_fraction(sec_values[i:i+my_window]) for i in range(0,len(sec_values),my_window)) # replace with your release number
                    sec_values = ''
        
                # chromosomes
                soft_mask= how_many(rec.seq,'acgt')
                ALL_all=how_many(rec.seq,'acgtACGT')
                GC_fraction=gc_fraction(rec.seq)
                chomosome_no += 1
                try:
                    GC_chromosoms[str(chomosome_no) + '+' + rec.id]=((GC_fraction,soft_mask/ALL_all),len(rec.seq))  # we distract N and n   
                except ZeroDivisionError:
                    GC_chromosoms[str(chomosome_no)+'+'+ rec.id]=((GC_fraction,0.0), len(rec.seq)) # distract N and n
        
                # count bases
                for base in 'acgtACGTNn':
                    if base in d_sec_values:
                        d_sec_values[base] += rec.seq.count(base)
                    else:
                        d_sec_values[base] = rec.seq.count(base)
                
                # graphs
                for w in range(0,len(rec.seq),my_window):
              
                    #print(range(i,i+window),len(gc_values_94[i:i+window]))
                    soft_mask= rec.seq[w:w+my_window].count('a') + rec.seq[w:w+my_window].count('t') + rec.seq[w:w+my_window].count('c') + rec.seq[w:w+my_window].count('g') 
                    ALL_all=soft_mask + rec.seq[w:w+my_window].count('A') + rec.seq[w:w+my_window].count('T') + rec.seq[w:w+my_window].count('C') + rec.seq[w:w+my_window].count('G')
                    GC_fraction=gc_fraction(rec.seq[w:w+my_window])
            
                    try:
                        GC_chromosoms_windowed[str(w)+'|'+ rec.id]=(GC_fraction,soft_mask/ALL_all)  # distract N and n    
                        #print(range(i,i+window),len(gc_values_94[i:i+window])) 
                    except ZeroDivisionError:
                        GC_chromosoms_windowed[str(w)+'|'+ rec.id]=(GC_fraction,0.0)  # distract N and n
                        
                # adjusted chromosome density
                #print(rec.id)                
            
                #fig = plt.figure(figsize=(int(50* (len(rec.seq)/max_len)),int(20* (len(rec.seq)/max_len))))
                fig = plt.figure(figsize=(50,20))
                ax1 = fig.add_subplot(111)
                
                ax1.use_sticky_edges = True
        
                names = list(GC_chromosoms_windowed.keys())
                
                v = list(GC_chromosoms_windowed.values())
                v1=[100 * i[0] for i in v]
                #v2=[i[1] for i in v]
                names_part=[n.split('|')[0] for n in names]             
                #values = np.array([special_value if (i[0]+i[1] == 0 and ALL_all== 0) else i[1]+slight_move if (i[1] => 0.0 and i[1] <= 0.00390625 and ALL_all > 0) else i[1] for i in v])  # values for special_indices
                values = np.array([special_value if i[0]+i[1] == 0 else i[1]+slight_move if (i[1] ==0 and i[0] != 0) else i[1] for i in v])  # values for special_indices
       
                if ALL_all== 0:
                    print('blue',rec.id,rec.seq)
                
                sc=ax1.scatter(names_part,v1, s=5, c=values,  cmap=new_cmap, marker="o", label='Blue dots are "NNnn" or spaces' ) # only GC fraction in y axe, only chromosomes
                
                ax1.grid(True)        
                
                #locator=MaxNLocator(prune='both',nbins=10) # Adjust the number of ticks as needed
                #ax1.yaxis.set_major_locator(locator)
                ax1.xaxis.set_ticks(np.arange(0, max_len//my_window, max_len//12000))   
                #ax1.set_xlim(0, max_len)
                #plt.xlim(0, max_len)
                
                #ax1.xaxis.set_major_formatter(StrMethodFormatter('{x:,}'))
                ax1.set_facecolor("lightgrey")   
        
                plt.title(f'GC% values of {my_fasta_file[1]} with {my_window} - chromosome {rec.id}', fontsize = 35) # replace the animal name and release no.
                plt.ylabel('GC fraction', fontsize = 30)
                plt.xlabel(f'Chromosome {rec.id} windows', fontsize = 30)
                #plt.xticks(np.arange(0, max_len, max_len//divide_by))
                plt.xticks(fontsize = 20, rotation = 30)
                plt.yticks(fontsize = 30)
                plt.legend(labelcolor='blue')
                
                cbar=plt.colorbar(sc,label="Red 0% soft-masked to green 100% soft-masked", orientation="horizontal")
        
                text = cbar.ax.xaxis.label
                font = font_manager.FontProperties(size=35)
                text.set_font_properties(font)        
        
                tick_font_size = 35
                cbar.ax.tick_params(labelsize=tick_font_size)
               
                
                plt.savefig(f'{where_i_am}{my_fasta_file[4]}/{my_fasta_file[1]}/{my_fasta_file[1]}_{my_window}_profile_soft_unmask_with_{rec.id}.png')  # replace the animal name and release no.
                #plt.show()
                plt.close('all')
       
                with open(f'{where_i_am}{my_fasta_file[4]}/{my_fasta_file[1]}/{my_fasta_file[1]}_{my_window}_profile_soft_unmask_{rec.id}.csv','w') as f: # replace the animal name and release no.
                    csv_file = csv.writer(f)
                    csv_file.writerows(GC_chromosoms_windowed.items())
               
               
        columns_dna = [column_name] # replace with your name e.g. release number - default GC_DNA
        gc_df_dna = pd.DataFrame(gc_values, columns = columns_dna) # replace with your release number

        #all_images = sorted([os.path.basename(f) for f in glob(f"{where_i_am}{my_fasta_file[1]}/*profile*.png")],key=lambda x: int(x.split('_')[-1].split('.')[0]))

        GC_chromosoms = OrderedDict(sorted(GC_chromosoms.items(), key=lambda x: x[1][1],reverse=True))

        #max_len=max(lengths)
    
        gc_df_dna.to_csv(f'{where_i_am}{my_fasta_file[4]}/{my_fasta_file[1]}/{my_fasta_file[1]}_per_windows_{my_window}.csv')
    
        #GC_chromosoms # replace with your release number
    
        with open(f'{where_i_am}{my_fasta_file[4]}/{my_fasta_file[1]}/{my_fasta_file[1]}_{my_window}.csv','w') as f:  # name csv file as you wish 
            for k,v in GC_chromosoms.items():
                f.write(f'{k.split("+")[0]},{k.split("+")[1]},{v[0][0]},{v[0][1]},{v[1]}\n')
                
        # scattered graphs - color by value, no display
        
        fig = plt.figure(figsize=(15,10))
        ax1 = fig.add_subplot(111)
        ax1.set_facecolor("lightgrey")  
        ax1.use_sticky_edges = True
        #ax1.margins(x=0.0,y=0.0)
        #print(range(i,i+window),len(gc_values_94[i:i+window]))
        
        names = list(i.split('+')[0] for i in GC_chromosoms.keys())
        v = list(GC_chromosoms.values())
        v1=[100 * i[0][0] for i in v]
        v2=[i[0][1] for i in v]

        sc=ax1.scatter(names,v1, s=150, c=v2, cmap='RdYlGn', marker="o", ) # only GC fraction in y axe, only chromosomes
        
        plt.grid(True)
        plt.title(f'GC% means of size-sorted chromosomes with {my_window} in {my_fasta_file[1]} ', fontsize = 18)  # Replace the animal name
        plt.ylabel('GC fraction', fontsize = 15)
        plt.xlabel('Chromosomes', fontsize = 15)
        plt.xticks(rotation = 30, fontsize = 12)
        plt.yticks(fontsize = 12)
        plt.colorbar(sc,label="Red 0% soft-masked to green 100% soft-masked", orientation="horizontal")
        plt.savefig(f'{where_i_am}{my_fasta_file[4]}/{my_fasta_file[1]}/{my_fasta_file[1]}_soft_unmask_per_chromosomes_{my_window}_{my_fasta_file[1]}.png')  # Replace the animal name
        #plt.show()
        plt.close('all')
        print(f'Done {my_fasta_file[1]}')
        
       
        

/home/marta/marta/marta/evan/PlosComp/dataOIST/Ansgar/Prochlorococcus_marinus/marinus_Anasgar.fasta 1000
Done Prochlorococcus_marinus


/tmp/ipykernel_6002/1129545953.py:25: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


/tmp/ipykernel_28093/3725709926.py:31: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
